In [ ]:
# ENS-SIZE folder should be deleted each time before Repeating the results 

In [1]:
import imbens
from imbens.datasets import make_imbalance
from imbens.utils._plot import plot_2Dprojection_and_cardinality
from imbens.ensemble.base import sort_dict_by_key
#from ucimlrepo import fetch_ucirepo ,list_available_datasets

import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import KernelPCA
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,average_precision_score,f1_score, matthews_corrcoef
from imblearn.metrics import geometric_mean_score


#import base models of SPE (KNN, DT, MLP, SVM, AdaBoost10, Bagging10, RandomForest10, GBDT10) 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn import ensemble


from imbens.ensemble import EasyEnsembleClassifier
from imbens.ensemble import BalanceCascadeClassifier
from imbens.ensemble import UnderBaggingClassifier
from imbens.ensemble import RUSBoostClassifier


# import DES algorithms for post ensemble selection 
from deslib.dcs import APosteriori
from deslib.dcs import APriori
from deslib.dcs import LCA
from deslib.dcs import OLA
from deslib.dcs import MCB
from deslib.dcs import MLA
from deslib.dcs import Rank


from deslib.des import DESP
from deslib.des import METADES
from deslib.des.knora_e import KNORAE
from deslib.des.knora_u import KNORAU
from deslib.des.knop import KNOP
from deslib.des.des_clustering import DESClustering
from deslib.des.des_knn import DESKNN
from deslib.des.des_mi import DESMI

from deslib.static.oracle import Oracle
from deslib.static.single_best import SingleBest
from deslib.static.stacked import StackedClassifier
from deslib.static.static_selection import StaticSelection


from deslib.des.probabilistic import DESKL
from deslib.des.probabilistic import Exponential
from deslib.des.probabilistic import Logarithmic
from deslib.des.probabilistic import MinimumDifference
from deslib.des.probabilistic import RRC

import os 
from os import walk
import warnings
from time import time
import numpy as np
import pandas as pd 
import colorama
from colorama import Fore, Style

from DESSPE import Hardness

from collections import Counter
from imblearn.datasets import fetch_datasets
from imblearn.over_sampling import ADASYN

In [3]:
warnings.filterwarnings("ignore")
#"KNN": KNeighborsClassifier(n_neighbors=5),
# "MLP": MLPClassifier(random_state=RANDOM_STATE, max_iter=200,hidden_layer_sizes=(128,10)),
path=os.getcwd()+"/keeldatasets/"
filenames = next(walk(path), (None, None, []))[2] 
os.mkdir('ENS-SIZE')
for j in range(len(filenames)):
    data=pd.read_csv(path+"{}".format(filenames[j]))
    df_obj = data.select_dtypes('object')
    data[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
    data.loc[data["Class"] == "positive", "Vclass"] = 1
    data.loc[data["Class"] == "negative", "Vclass"] = int(0)
    data["Vclass"] = data["Vclass"].apply(np.int32)
    y=data.Vclass
    X=data.drop(columns=["Class","Vclass"],axis=1)
    
    

    Results= {}
    Results["dname"]= filenames[j].strip(".dat")
    output = pd.DataFrame() 
    for i in range(15):
        Results["iteration"]= i+1
        base_estimators={
               "DT": DecisionTreeClassifier(max_depth=10),}
               #"SVM": SVC(probability=True,C=1000)}
               #"AdaBoost10": AdaBoostClassifier(n_estimators=10, random_state=i)}
               #"Bagging10":  BaggingClassifier(n_estimators=10, random_state=i),
               #"RandomForest10": RandomForestClassifier(n_estimators=10, random_state=i),
               #"GBDT10": ensemble.GradientBoostingClassifier(n_estimators=10,random_state=i)}

        print("Shape of " +filenames[j] ,X.shape)
    
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=i,stratify=y)
    
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        
        
        # Forming balanced DSEL from X_train and y_train via ADASYN  
        X_dsel, y_dsel = ADASYN().fit_resample(X_train, y_train)
    
    
        origin_distr = sort_dict_by_key(Counter(y))
        train_distr=sort_dict_by_key(Counter(y_train))
        dsel_distr=sort_dict_by_key(Counter(y_dsel))
        test_distr = sort_dict_by_key(Counter(y_test))
        print("Imbalance Ratio:", origin_distr[0]/origin_distr[1])
        print('Original dataset shape %s' % origin_distr)
        print('Train dataset shape %s' % train_distr)
        print('DSEL dataset shape %s' % dsel_distr)
        print('Test dataset shape %s' % test_distr)
        print("-------------------------------------")

           
        for base_name,base_model in base_estimators.items():
                
            Results["base_model"]= base_name
            # SPE with different ensemble sizes
            for size in [10,20,40]:
                Results["size"]= size
                clf = imbens.ensemble.SelfPacedEnsembleClassifier(estimator=base_model, 
                                                n_estimators=size).fit(X_train, y_train,  train_verbose={'print_distribution': False},
                                                                     eval_datasets= {'test': (X_test, y_test)})
        
        
                # Under_Sampling Ensemble Algorithms to comapre against SPE and SPE++
                Easy= EasyEnsembleClassifier(estimator=base_model, n_estimators=size, random_state=i)
                Cascade= BalanceCascadeClassifier(estimator=base_model, n_estimators=size, random_state=i)
                RusBoost= RUSBoostClassifier(estimator=base_model, n_estimators=size, random_state=i)
        
                
                # Imbalanced learning aglorithms 
                
                imbmodels_names = ["SPE", "Easy", "Cascade","RusBoost"]
                                 
                imbmodels=[clf,Easy,Cascade,RusBoost]
        
                for i in range(len(imbmodels)):
                    
                    Results["Algorithm"]=imbmodels_names[i]
                    
                    start_time = time()
                    imbmodels[i].fit(X_train, y_train)  
                    Results["Time"]=  time() - start_time
                    imb_propa = imbmodels[i].predict_proba(X_test)
                    imb_propa = imb_propa[:,1] 
                    imb_label = imbmodels[i].predict(X_test) 
                    Results["AUROC"] = roc_auc_score(y_test, imb_propa)
                    Results["PRAUC"]= average_precision_score(y_test,imb_propa)
                    Results["F1_score"]= f1_score(y_test,imb_label)
                    Results["geometric_mean"]= geometric_mean_score(y_test,imb_label)
                    Results["matthews_corrcoef"]= matthews_corrcoef(y_test, imb_label)
                        
                    df_dictionary = pd.DataFrame([Results])
                    output = pd.concat([output, df_dictionary], ignore_index=True)
                   
                
                
                desp = DESP(clf)
                knorau = KNORAU(clf)
                desKNN = DESKNN(clf)
                knop = KNOP(clf)
                meta = METADES(clf)
                knorae = KNORAE(clf)
                deskl =  DESKL(clf)
                rrc = RRC(clf)
                
                
                
                list_ds_names = [  'DES-P', 'KNORA-U', "DES-KNN",'KNOP', 'META-DES','KNORA-E',
                             'DES-KL', 'DES-RRC'   ]
            
                list_ds=[desp, knorau, desKNN, knop,   meta, knorae, deskl,rrc]
                
                
                for i in range(len(list_ds)):
                    
                    
                    Results["Algorithm"]=list_ds_names[i]
                    start_time = time()
                    list_ds[i].fit(X_dsel, y_dsel)    
                    
                   
                    Results["Time"]=  time() - start_time
                    des_proba= list_ds[i].predict_proba(X_test)
                    des_label= list_ds[i].predict(X_test)
                    des_proba= des_proba[:,1]
    
                
                    pos_nan=np.argwhere(np.isnan(des_proba))
                    if(len(pos_nan)):
                        pos=np.unique(pos_nan[:,0])
                        des_proba[pos]= (des_label[pos]==1)*1
    
                    
                    Results["AUROC"] = roc_auc_score(y_test, des_proba)
                    Results["PRAUC"]= average_precision_score(y_test,des_proba)
                    Results["F1_score"]= f1_score(y_test,des_label)
                    Results["geometric_mean"]= geometric_mean_score(y_test,des_label)
                    Results["matthews_corrcoef"]= matthews_corrcoef(y_test, des_label)
                    df_dictionary = pd.DataFrame([Results])
                    output = pd.concat([output, df_dictionary], ignore_index=True)
         
           
    output.to_csv(r'ENS-SIZE/{}.csv'.format(filenames[j]), index=False)        

Shape of cleveland-0_vs_4.dat (173, 13)
Imbalance Ratio: 12.307692307692308
Original dataset shape {0: 160, 1: 13}
Train dataset shape {0: 128, 1: 10}
DSEL dataset shape {0: 128, 1: 127}
Test dataset shape {0: 32, 1: 3}
-------------------------------------
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃             ┃            Data: train             ┃             Data: test             ┃
┃ #Estimators ┃               Metric               ┃               Metric               ┃
┃             ┃  acc    balanced_acc   weighted_f1 ┃  acc    balanced_acc   weighted_f1 ┃
┣━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┫
┃      1      ┃ 0.696      0.790          0.769    ┃ 0.686      0.375          0.744    ┃
┃      2      ┃ 0.942      0.923          0.948    ┃ 0.886      0.484          0.859    ┃
┃      3      ┃ 0.899      0.945          0.917    ┃ 0.943      0.818          0.943    ┃
┃      4      ┃ 0.949 

# Analyzing the effect of ensemble size

In [21]:
import pandas as pd 
import os 
from os import walk

In [22]:
# Read files from output directory and list their names
path=os.getcwd()+"\ENS-SIZE"
print(path)

filenames = next(walk(path), (None, None, []))[2] 
print(filenames)

C:\Users\User\Comp Poland\mesa-Dynamic Selection\CODE-DSPE\ENS-SIZE
['cleveland-0_vs_4.dat.csv', 'ecoli-0-3-4_vs_5.dat.csv', 'ecoli-0-6-7_vs_3-5.dat.csv', 'glass-0-1-5_vs_2.dat.csv', 'glass4.dat.csv', 'yeast-2_vs_4.dat.csv']


In [23]:
# Combine results of all files into Combined_{DT,SVM,AdaBoost10} based on base estimator of SPE 

combined=pd.read_csv("ENS-SIZE/{}".format(filenames[0]))
combined.head()
for i in range(1,len(filenames)):
    new=pd.read_csv("ENS-SIZE/{}".format(filenames[i]))
    combined=combined.append(new, ignore_index = True)
combined.to_csv("ENS-SIZE/Combined.csv")

DT_10= combined[combined['size']==10]
DT_10.to_csv("ENS-SIZE/Combined_10.csv", index=False) 

DT_20= combined[combined['size']==20]
DT_20.to_csv("ENS-SIZE/Combined_20.csv", index=False) 

DT_40= combined[combined['size']==40]
DT_40.to_csv("ENS-SIZE/Combined_40.csv", index=False)

# Results for the ensemble size = 10

## Repeat the next 2 cells for {Combined_20.csv, Combined_40.csv}

In [24]:
data=pd.read_csv("ENS-SIZE/Combined_10.csv")
print(data.shape)
print("# Algorithms ", data.Algorithm.nunique())
print("# Models", data.base_model.nunique())
print("# Iterations", data.iteration.nunique()) 
data.head(31)

(1080, 11)
# Algorithms  12
# Models 1
# Iterations 15


,dname,iteration,base_model,size,Algorithm,Time,AUROC,PRAUC,F1_score,geometric_mean,matthews_corrcoef
0,cleveland-0_vs_4,1,DT,10,SPE,0.051828,0.979167,0.733333,0.750000,0.968246,0.750000
1,cleveland-0_vs_4,1,DT,10,Easy,0.048189,0.942708,0.555556,0.500000,0.777282,0.458333
2,cleveland-0_vs_4,1,DT,10,Cascade,0.044535,0.953125,0.642857,0.400000,0.568258,0.364326
3,cleveland-0_vs_4,1,DT,10,RusBoost,0.052273,0.963542,0.722222,0.666667,0.803638,0.635417
4,cleveland-0_vs_4,1,DT,10,DES-P,0.009338,0.937500,0.477778,0.750000,0.968246,0.750000
5,cleveland-0_vs_4,1,DT,10,KNORA-U,0.012739,0.937500,0.533333,0.750000,0.968246,0.750000
6,cleveland-0_vs_4,1,DT,10,DES-KNN,0.012066,0.937500,0.500000,0.666667,0.951972,0.673146
7,cleveland-0_vs_4,1,DT,10,KNOP,0.019008,0.947917,0.533333,0.666667,0.951972,0.673146
8,cleveland-0_vs_4,1,DT,10,META-DES,0.072300,0.958333,0.700000,0.750000,0.968246,0.750000
9,cleveland-0_vs_4,1,DT,10,KNORA-E,0.012006,0.765625,0.473016,0.571429,0.790569,0.531596


In [25]:
filtered=data.groupby(['dname',"Algorithm"])
output=filtered.mean()


datasets=data["dname"].unique()
PRAUC_AllDT=pd.DataFrame({"PRAUC":output.loc[datasets[0],"PRAUC"]}).transpose()
PRAUC_AllDT.insert(0, "name", datasets[0], True)
for i in range(1,len(datasets)):
    temp=pd.DataFrame({"PRAUC":output.loc[datasets[i],"PRAUC"]}).transpose()
    temp.insert(0, "name", datasets[i], True)
    PRAUC_AllDT=PRAUC_AllDT.append(temp)
final=PRAUC_AllDT[["name",'SPE','Easy','Cascade','RusBoost','DES-P','KNORA-U','DES-KNN','KNOP','META-DES','KNORA-E','DES-KL','DES-RRC' ]]

final

Algorithm,name,SPE,Easy,Cascade,RusBoost,DES-P,KNORA-U,DES-KNN,KNOP,META-DES,KNORA-E,DES-KL,DES-RRC
PRAUC,cleveland-0_vs_4,0.700096,0.652721,0.553615,0.640025,0.706881,0.784301,0.578862,0.720820,0.689418,0.596085,0.706164,0.706164
PRAUC,ecoli-0-3-4_vs_5,0.920542,0.901418,0.914156,0.920003,0.976190,0.981111,0.886852,0.945556,0.931389,0.933968,0.960079,0.960079
PRAUC,ecoli-0-6-7_vs_3-5,0.788877,0.785720,0.766779,0.752556,0.729616,0.817275,0.606786,0.805019,0.810796,0.596971,0.818680,0.818680
PRAUC,glass-0-1-5_vs_2,0.402864,0.334448,0.334135,0.356024,0.463250,0.515547,0.357301,0.495432,0.418372,0.373810,0.416984,0.416984
PRAUC,glass4,0.746905,0.706854,0.762450,0.675507,0.845519,0.874550,0.719138,0.767430,0.777214,0.812429,0.826064,0.826064
PRAUC,yeast-2_vs_4,0.798543,0.753374,0.773418,0.785747,0.751303,0.799662,0.676631,0.780813,0.738304,0.687888,0.805243,0.805243
